# Example for Single Quantum SNSPD

In [ ]:
import qcodes as qc
import matplotlib.pyplot as plt

from qcodes_contrib_drivers.drivers.SingleQuantum.SingleQuantum import WebSQControlqcode
from qcodes.instrument.parameter import expand_setpoints_helper
from qcodes.dataset.plotting import plot_dataset
from qcodes.logger.logger import start_all_logging

Connects to the driver, default ip address is 192.168.1.1

In [ ]:
start_all_logging()
station = qc.Station()
websq = WebSQControlqcode('websq', address='192.168.1.1', port=12000)
station.add_component(websq)

Get the number of detectors

In [ ]:
number_of_detectors = websq.number_of_detectors()

Set the integration time (ms)

In [ ]:
websq.measurement_periode(20)

Enable the detectors

In [ ]:
websq.detectors(True)

Set the bias current (uA) and trigger levels (mV) with array size equal to the number of detectors in your system.

In [ ]:
websq.bias_current([1, 2, 3, 4])
websq.trigger_level([5, 6, 7, 8])

Set the amount of measurements

In [ ]:
websq.npts(5)

Measurement of a simple IV sweep

In [ ]:
exp = qc.load_or_create_experiment(
    experiment_name='performing_meas_using_parameters_and_dataset',
    sample_name="no sample")
timemeas = qc.Measurement(exp=exp)
timemeas.register_parameter(websq.channel1)
timemeas.register_parameter(websq.channel2)
timemeas.register_parameter(websq.channel3)
timemeas.register_parameter(websq.channel4)

with timemeas.run() as datasaver:
    for i in range(10):
        # Tests increasing the bias current
        websq.bias_current([i, i, i, i])
        # Updates the detector counts
        websq.counters()
        datasaver.add_result(
            *expand_setpoints_helper(websq.channel1),
            *expand_setpoints_helper(websq.channel2),
            *expand_setpoints_helper(websq.channel3),
            *expand_setpoints_helper(websq.channel4)
        )

Close the connection and plot the measurement

In [ ]:
dataset = datasaver.dataset
fig, ax = plt.subplots(1)
axes, cbaxes = plot_dataset(dataset, axes=[ax, ax, ax, ax])
# Close connection
websq.close()
plt.show()